In [13]:
import pandas as pd

file_path = '../data/MachineLearningRating_v3.txt'  
data = pd.read_csv(file_path, delimiter='|')  


C:\Users\arsem\AppData\Local\Temp\ipykernel_21628\3832553365.py:4: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, delimiter='|')


In [14]:
print(data.dtypes)


UnderwrittenCoverID           int64
PolicyID                      int64
TransactionMonth             object
IsVATRegistered                bool
Citizenship                  object
LegalType                    object
Title                        object
Language                     object
Bank                         object
AccountType                  object
MaritalStatus                object
Gender                       object
Country                      object
Province                     object
PostalCode                    int64
MainCrestaZone               object
SubCrestaZone                object
ItemType                     object
mmcode                      float64
VehicleType                  object
RegistrationYear              int64
make                         object
Model                        object
Cylinders                   float64
cubiccapacity               float64
kilowatts                   float64
bodytype                     object
NumberOfDoors               

In [15]:
print(data['TotalClaims'].describe())
print(data['TotalClaims'].value_counts(dropna=False).head(10))


count    1.000098e+06
mean     6.486119e+01
std      2.384075e+03
min     -1.200241e+04
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      3.930921e+05
Name: TotalClaims, dtype: float64
TotalClaims
0.000000        997305
6140.350877        326
750.649123         101
1300.000000         59
850.000000          51
877.192982          45
43859.649123        41
1150.000000         31
3070.175439         28
815.789474          24
Name: count, dtype: int64


In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Step 1: Subset to positive TotalClaims
df_model = data[data['TotalClaims'] > 0].copy()

# Step 2: Drop outliers in TotalClaims
df_model = df_model[df_model['TotalClaims'] <= df_model['TotalClaims'].quantile(0.99)]

# Step 3: Separate target and features
y = df_model['TotalClaims']
X = df_model.drop(columns=['TotalClaims'])

# Step 4: Replace empty strings and strip spaces
X = X.replace(r'^\s*$', np.nan, regex=True)
X = X.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Step 5: Encode categoricals
X = pd.get_dummies(X, drop_first=True)

# Step 6: Drop columns with too many missing values (optional but safe)
null_pct = X.isnull().mean()
X = X.loc[:, null_pct < 0.5]

# Step 7: Fill remaining NaNs with median
X = X.fillna(X.median(numeric_only=True))

# Final check: are there still NaNs?
if X.isna().sum().sum() > 0:
    print("❌ Still has NaNs:")
    print(X.isna().sum()[X.isna().sum() > 0])
    raise ValueError("Fix these columns manually.")
else:
    print("✅ All missing values handled.")

# Step 8: Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 9: Train model
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Step 10: Evaluate
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"✅ Model trained successfully.\nRMSE: {rmse:.2f}\nR²: {r2:.4f}")


C:\Users\arsem\AppData\Local\Temp\ipykernel_21628\4227126243.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X = X.applymap(lambda x: x.strip() if isinstance(x, str) else x)


✅ All missing values handled.
✅ Model trained successfully.
RMSE: 26014.23
R²: 0.2949


✅ Model Summary (Claim Severity Prediction)
Target: TotalClaims (for records where claims > 0)

Model: Linear Regression

Result:

RMSE: 26,014.23
→ On average, your model’s prediction is off by ~26K ZAR.

R²: 0.2949
→ ~29.5% of the variation in TotalClaims is explained by your features.

📊 Interpretation
This R² score is modest — it suggests the model captures some structure, but there's a lot of noise or missing signal.

This is common with linear models on complex financial data.

The next step will be to compare this with more powerful models like:

Random Forest

XGBoost

In [26]:
from sklearn.ensemble import RandomForestRegressor

# Step 1: Initialize the model
rf_model = RandomForestRegressor(
    n_estimators=100, 
    max_depth=None, 
    random_state=42,
    n_jobs=-1
)

# Step 2: Train the model
rf_model.fit(X_train, y_train)

# Step 3: Predict on test set
y_pred_rf = rf_model.predict(X_test)

# Step 4: Evaluate
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

print(f"🌲 Random Forest Model Results:")
print(f"✅ RMSE: {rmse_rf:.2f}")
print(f"✅ R²: {r2_rf:.4f}")


🌲 Random Forest Model Results:
✅ RMSE: 26543.08
✅ R²: 0.2660


In [32]:
# Step 1: Identify categorical columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
print("Categorical columns:", categorical_cols)


Categorical columns: []


In [33]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)

rf_rmse = np.sqrt(mean_squared_error(y_test, rf_preds))
rf_r2 = r2_score(y_test, rf_preds)

print(f"Random Forest RMSE: {rf_rmse:.2f}")
print(f"Random Forest R²: {rf_r2:.4f}")


Random Forest RMSE: 26543.08
Random Forest R²: 0.2660
